In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data

data1  =  pd.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet\partition_id=7/part-0.parquet')
data2  =  pd.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet\partition_id=8/part-0.parquet')
data3  =  pd.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet\partition_id=9/part-0.parquet')

In [3]:
# remove the nan values
def replace_nan_with_mean_or_zero(column):
    mean_value = column.mean()
    if pd.notnull(mean_value):
        return column.fillna(mean_value)
    else:
        return column.fillna(0)

In [4]:
train_data = pd.concat([ data1,data2,data3], axis=0)  # 3 partitions
del  data1, data2, data3 # free up memory

In [5]:
train_data.apply(replace_nan_with_mean_or_zero)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,1190,0,0,2.888326,0.928983,0.441290,1.352455,1.301461,2.302733,-0.231061,...,-0.203992,-0.257635,-0.254294,-0.100387,-0.390012,-1.553783,0.224214,-0.278927,-1.170454,0.546211
1,1190,0,1,5.419313,1.793786,0.198769,1.165048,0.392095,2.187486,-0.220963,...,-0.289453,0.184713,0.047625,-0.074191,-0.458309,-1.218235,-0.594166,-0.541755,-1.621669,-0.907372
2,1190,0,2,1.286522,1.177853,0.071345,1.043871,1.246540,1.825975,-0.283726,...,-0.304391,-0.082534,-0.153695,-0.007306,1.986761,0.009597,0.996386,3.743622,0.111650,1.000657
3,1190,0,3,1.223723,1.093739,-0.049899,0.715656,0.709871,1.136459,-0.194208,...,1.530262,-0.969038,-1.313207,-1.890294,2.067578,3.928346,0.250758,3.072302,5.000000,2.265868
4,1190,0,4,1.949932,1.186918,0.294662,0.873467,0.810031,2.020650,-0.113065,...,-0.140796,0.380849,0.089629,1.338810,-1.736215,0.047372,-0.504546,-2.281883,0.032377,-2.002821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274571,1698,967,34,3.242493,2.525160,-0.721981,2.544025,2.477615,0.417557,0.785812,...,0.016936,0.243475,0.166927,0.384940,-0.174297,-0.066046,-0.038767,-0.132337,-0.022426,-0.252461
6274572,1698,967,35,1.079139,1.857906,-0.790646,2.745439,2.339877,0.845065,0.651370,...,0.050860,0.850152,0.909382,1.015314,0.235962,0.122539,0.099559,-0.249584,-0.123571,-0.460630
6274573,1698,967,36,1.033172,2.515527,-0.672298,2.289250,2.521592,0.255077,0.919892,...,0.152333,0.395684,-0.292574,-3.215846,-0.535129,-0.178484,-1.808150,-0.065355,-0.000367,-0.125170
6274574,1698,967,37,1.243116,2.663298,-0.889112,2.313155,3.101428,0.324454,0.618944,...,-0.029483,1.925987,0.479394,3.621867,-0.107114,-0.063599,1.204755,-0.148711,-0.026583,-0.256395


In [6]:
# define the columns
lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]

In [7]:
#new lag data
lags = train_data[lag_cols_original]
lags = lags.rename(columns= { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)})

lags.date_id = lags.date_id + 1
lags = lags.groupby(['date_id', 'symbol_id']).last().reset_index()  # pick up last record of previous date
train_data = train_data.merge(lags, on = ['date_id', 'symbol_id'], how = 'left')
del lags ,lag_cols_original
train_data =  train_data.fillna(0) 

In [8]:
train_data.reset_index(drop=True)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_8,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,1190,0,0,2.888326,0.928983,0.441290,1.352455,1.301461,2.302733,-0.231061,...,0.546211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1190,0,1,5.419313,1.793786,0.198769,1.165048,0.392095,2.187486,-0.220963,...,-0.907372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1190,0,2,1.286522,1.177853,0.071345,1.043871,1.246540,1.825975,-0.283726,...,1.000657,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1190,0,3,1.223723,1.093739,-0.049899,0.715656,0.709871,1.136459,-0.194208,...,2.265868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1190,0,4,1.949932,1.186918,0.294662,0.873467,0.810031,2.020650,-0.113065,...,-2.002821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18750155,1698,967,34,3.242493,2.525160,-0.721981,2.544025,2.477615,0.417557,0.785812,...,-0.252461,0.501321,0.905332,-0.819582,-0.564046,-0.223018,-0.283954,-0.045938,0.009797,-0.102538
18750156,1698,967,35,1.079139,1.857906,-0.790646,2.745439,2.339877,0.845065,0.651370,...,-0.460630,-1.113053,0.697190,-1.619031,-1.222743,-0.706082,-0.291133,0.167733,0.099704,0.324610
18750157,1698,967,36,1.033172,2.515527,-0.672298,2.289250,2.521592,0.255077,0.919892,...,-0.125170,-1.019353,-0.460962,-2.026678,-0.848606,-0.305448,-1.256913,-0.109359,-0.027474,-0.253956
18750158,1698,967,37,1.243116,2.663298,-0.889112,2.313155,3.101428,0.324454,0.618944,...,-0.256395,0.235850,0.556479,0.618944,-0.243765,-0.108361,-0.260777,-0.486923,-0.275566,-1.020708


In [10]:
train_data.to_parquet('data/df.parquet')

In [11]:
num_val = 1670
val_set = train_data[train_data.date_id >= num_val]
val_set.to_parquet('data/validate_data.parquet')

In [15]:
train_data.drop(train_data.loc[train_data.date_id >= num_val].index, inplace=True)

train_data.to_parquet('data/train_data.parquet')